In [1]:
import os
import pickle
from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import ProcessPoolExecutor, as_completed
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import sys
sys.path.append(str(Path.cwd().parent))

In [2]:
# Directory containing pickle files
directory = Path("../results/classification/quantitative")

# Optionally filter for specific substrings
pkl_files = [f for f in directory.iterdir() if f.suffix == ".pkl"]

# Efficient loader
def load_file(file_path):
    try:
        with open(file_path, "rb") as f:
            data = pickle.load(f)
            if isinstance(data, dict):
                return file_path.name, data
    except (EOFError, pickle.UnpicklingError) as e:
        print(f"Warning: Failed to load {file_path.name}: {e}")
    except Exception as e:
        print(f"Unexpected error with {file_path.name}: {e}")
    return None

# Load files using multithreading (I/O bound)
all_data = {}
with ThreadPoolExecutor() as executor:
    futures = [executor.submit(load_file, f) for f in pkl_files]
    for future in as_completed(futures):
        result = future.result()
        if result:
            filename, data = result
            all_data[filename] = data

files = list(all_data.keys())


In [3]:
records = []

for file, dictionary in all_data.items():
    records.append(dictionary)

# Create the DataFrame
df = pd.DataFrame(records)

print(df.shape)

(10011, 15)


In [4]:
df_long = df.melt(
        id_vars=['name', 'random_state', 'prox_method', 'conformity_k'],
        value_vars=[
                'ice_auc', 'diff_proba_auc', 'conformity_auc',
                'ice_auc_test', 'diff_proba_auc_test', 'conformity_auc_test',
                'tree_conformity_auc', 'tree_conformity_auc_test'
        ],
        var_name='metric',
        value_name='auc'
)

In [5]:
df_long['Method'] = df_long['metric']

df_long.loc[df_long['metric'] == 'ice_auc', 'Method'] = 'RF-ICE (EMR)'
df_long.loc[(df_long['metric'] == 'conformity_auc') & (df_long['prox_method'] == 'original'), 'Method'] = 'Conformity'
df_long.loc[(df_long['metric'] == 'conformity_auc') & (df_long['prox_method'] == 'rfgap'), 'Method'] = 'RF-ICE Conformity'
df_long.loc[df_long['metric'] == 'diff_proba_auc', 'Method'] = 'Proba. Diff.'
df_long.loc[df_long['metric'] == 'tree_conformity_auc', 'Method'] = 'Tree Conformity'


methods = ['RF-ICE (EMR)', 'Conformity', 'RF-ICE Conformity', 'Proba. Diff.', 'Tree Conformity']
df_long = df_long[df_long['Method'].isin(methods)]


In [6]:
df_group = df_long.groupby(['prox_method', 'conformity_k', 'metric'])['auc'].agg(['mean', 'std']).reset_index()

df_group.loc[
    df_group['metric'].isin([
        'diff_proba_auc', 'diff_proba_auc_test',
        'tree_conformity_auc', 'tree_conformity_auc_test'
    ]),
    'prox_method'
] = 'NA'


# Subset for the compared methods


df_group['Method'] = df_group['metric']

df_group.loc[df_group['metric'] == 'ice_auc', 'Method'] = 'RF-ICE (EMR)'
df_group.loc[(df_group['metric'] == 'conformity_auc') & (df_group['prox_method'] == 'original'), 'Method'] = 'Conformity'
df_group.loc[(df_group['metric'] == 'conformity_auc') & (df_group['prox_method'] == 'rfgap'), 'Method'] = 'RF-ICE Conformity'
df_group.loc[df_group['metric'] == 'diff_proba_auc', 'Method'] = 'Proba. Diff.'
df_group.loc[df_group['metric'] == 'tree_conformity_auc', 'Method'] = 'Tree Conformity'

# Carefull with this one!
methods = ['RF-ICE (EMR)', 'Conformity', 'RF-ICE Conformity', 'Proba. Diff.', 'Tree Conformity']
df_group = df_group[df_group['Method'].isin(methods)]
# df_group.drop(columns=['prox_method', 'conformity_k', 'metric'], inplace=True)

df_group.reset_index(drop = True, inplace=True)

In [7]:
# df_group = df_group[df_group['conformity_k'] > 10]

In [8]:
df_latex = df_group.groupby('Method')[['mean', 'std']].mean().sort_values('mean', ascending=False).reset_index()
df_latex['AUC'] = df_latex.apply(lambda row: f"{row['mean']:.3f} ± {row['std']:.3f}", axis=1)
latex_table = df_latex[['Method', 'AUC']].to_latex(index=False)
print(latex_table)


\begin{tabular}{ll}
\toprule
Method & AUC \\
\midrule
RF-ICE Conformity & 0.965 ± 0.068 \\
Conformity & 0.964 ± 0.069 \\
RF-ICE (EMR) & 0.932 ± 0.099 \\
Proba. Diff. & 0.929 ± 0.118 \\
Tree Conformity & 0.730 ± 0.196 \\
\bottomrule
\end{tabular}

